In [1]:
SAMPLE_TEXT = """
be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe
edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc
fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg
fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb
aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea
fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb
dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe
bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef
egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb
gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce
"""

In [14]:
from collections import defaultdict

In [2]:
def tokenize_line(line):
    first, second = line.split(" | ")
    return first.split(" "), second.split(" ")

def parse_text(raw_text):
    return [tokenize_line(l) for l in raw_text.split("\n") if l]

def read_input():
    with open("input.txt", "rt") as f:
        return f.read()

In [9]:
def count_special_characters(lines):
    # 1 - 2 segments
    # 7 - 3 segments
    # 4 - 4 segments
    # 8 - 7 segments
    count = 0
    for _, second in lines:
        count += len([x for x in second if len(x) in (2, 3, 4, 7)])
    return count

In [5]:
count_special_characters(parse_text(SAMPLE_TEXT))

26

In [6]:
count_special_characters(parse_text(read_input()))

530

In [82]:
def decode_segments(line):
    """
    Segment numbers:
    - 0 - top
    - 1 - top-left
    - 2 - top-right
    - 3 - middle
    - 4 - bottom-left
    - 5 - bottom-right
    - 6 - bottom

    Numbers
    1 - 2 segments
    7 - 3 segments
    4 - 4 segments
    2, 3, 5 - 5 segments
    0, 6, 9- 6 segments
    8 - 7 segments
    """
    segments = [None] * 7
    length_map = defaultdict(list)
    for group in line[0]:
        length_map[len(group)].append(group)

    one = length_map[2][0]
    seven = length_map[3][0]
    four = length_map[4][0]
    two_three_five = length_map[5]
    three = [c for c in two_three_five if set(c) & set(one) == set(one)][0]

    segments[0] = set(seven) - set(one)
    segments[3] = (set(three) - set(one)) & set(four)
    segments[6] = set(three) - set(four) - segments[0]
    segments[1] = set(four) - set(one) - segments[3]

    five = [c for c in two_three_five if set(c) & segments[1] == segments[1]][0]

    segments[5] = set(five) & set(one)
    segments[2] = set(one) - segments[5]

    two = [c for c in two_three_five if c not in (three, five)][0]

    segments[4] = set(two) - set(three)
    return [s.pop() for s in segments]

def calc_digit(characters, segment_mapping):
    segment_map = {
        (1, 1, 1, 0, 1, 1, 1): "0",
        (0, 0, 1, 0, 0, 1, 0): "1",
        (1, 0, 1, 1, 1, 0, 1): "2",
        (1, 0, 1, 1, 0, 1, 1): "3",
        (0, 1, 1, 1, 0, 1, 0): "4",
        (1, 1, 0, 1, 0, 1, 1): "5",
        (1, 1, 0, 1, 1, 1, 1): "6",
        (1, 0, 1, 0, 0, 1, 0): "7",
        (1, 1, 1, 1, 1, 1, 1): "8",
        (1, 1, 1, 1, 0, 1, 1): "9",
    }
    segments = tuple(1 if s in characters else 0 for s in segment_mapping)
    return segment_map[segments]


def calc_digits(line, segment_mapping):
    return "".join(calc_digit(c, segment_mapping) for c in line[1])


def calc_value_of_line(line):
    mapping = decode_segments(line)
    result = int(calc_digits(line, mapping))
    return result

def sum_lines(lines):
    return sum(calc_value_of_line(l) for l in lines)

In [83]:
sum_lines(parse_text(SAMPLE_TEXT))

61229

In [84]:
sum_lines(parse_text(read_input()))

1051087